# Conexión

In [1]:
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
import pandas as pd
import numpy as np
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display
from sqlalchemy.engine import URL
import ast

In [2]:
USER = "slqleo"
PWD  = "G?,6(4(q$GMw"          # sin escaparla, URL.create se encarga
HOST = "45.169.253.115"       # o el host que realmente tenga la DB
PORT = 3306

srv_url = URL.create(
    drivername="mysql+pymysql",
    username=USER,
    password=PWD,
    host=HOST,
    port=PORT,
    query={"charset": "utf8mb4"},
)
engine_srv = create_engine(srv_url, pool_pre_ping=True)

with engine_srv.connect() as conn:
    server = conn.execute(text("SELECT @@hostname, @@port, @@version")).one()
    print("Conectado a:", server)

Conectado a: ('serverbd', 3306, '8.0.44-0ubuntu0.22.04.1')


# Usabilidad

In [3]:
with engine_srv.connect() as conn:
    usability_events = pd.read_sql(
        text("SELECT * FROM slq_main.usability_events"),
        conn
    )

usability_events.head(2)

,id,function_used,slq_company_id,sys_user_id,acl_role_id,created_at,updated_at
0,1,Visualización del detalle de un producto,168,2146.0,1.0,2025-03-26 21:10:48,2025-03-26 21:10:48
1,2,Visualización del detalle de un producto,168,2146.0,1.0,2025-03-26 21:11:08,2025-03-26 21:11:08


In [4]:
query = text("""
SELECT 
    u.function_used,
    u.created_at,
    u.sys_user_id AS usuario,
    e.nombre AS empresa
FROM slq_main.usability_events u
INNER JOIN slq_main.slq_empresas e 
    ON u.slq_company_id = e.id
LEFT JOIN slq_main.sys_users ur 
    ON u.sys_user_id = ur.id
WHERE 
    e.activo = 1
    AND ur.activo = 1
    AND ur.interno = 0
    AND e.nombre NOT REGEXP 'Mejora|academy|prueba|Demostración'
""")

with engine_srv.connect() as conn:
    df_usuarios = pd.read_sql(query, conn)

df_usuarios

,function_used,created_at,usuario,empresa
0,Acceso al módulo de productos,2025-03-27 08:01:32,26,CAJA DE COMPENSACIÓN FAMILIAR COMPENSAR
1,Visualización del detalle de un producto,2025-03-27 08:02:05,26,CAJA DE COMPENSACIÓN FAMILIAR COMPENSAR
2,Descarga de etiquetas,2025-03-27 08:02:32,26,CAJA DE COMPENSACIÓN FAMILIAR COMPENSAR
3,Acceso al módulo de productos,2025-03-27 08:16:25,26,CAJA DE COMPENSACIÓN FAMILIAR COMPENSAR
4,Visualización del detalle de un producto,2025-03-27 08:17:02,26,CAJA DE COMPENSACIÓN FAMILIAR COMPENSAR
...,...,...,...,...
151418,Visualización del detalle de un producto,2025-12-04 11:48:20,3176,LEWIS ENERGY COLOMBIA INC
151419,Ingreso a login,2025-12-04 11:48:45,3176,LEWIS ENERGY COLOMBIA INC
151420,Ingreso a login,2025-12-04 11:49:23,3176,LEWIS ENERGY COLOMBIA INC
151421,Visualización del detalle de un producto,2025-12-04 11:50:50,3176,LEWIS ENERGY COLOMBIA INC


In [5]:
df_usuarios.to_csv("empresas.csv")

# Productos

In [9]:
query = text("""
WITH audit_crea AS (
    SELECT 
        a.auditable_id,
        COALESCE(a.usu_id, a.user_id) AS user_id,
        ROW_NUMBER() OVER (
            PARTITION BY a.auditable_id
            ORDER BY a.changed_at ASC, a.id ASC
        ) AS rn
    FROM slq_main.audits a
    WHERE 
        a.auditable_type = 'App\\\\Http\\\\ApiCore\\\\Productos\\\\ProdProducto'
        AND a.event = 'created'
),

audit_upd AS (
    SELECT 
        a.auditable_id,
        COALESCE(a.usu_id, a.user_id) AS user_id,
        a.changed_at,
        ROW_NUMBER() OVER (
            PARTITION BY a.auditable_id
            ORDER BY a.changed_at DESC, a.id DESC
        ) AS rn
    FROM slq_main.audits a
    WHERE 
        a.auditable_type = 'App\\\\Http\\\\ApiCore\\\\Productos\\\\ProdProducto'
        AND a.event = 'updated'
),

adjuntos AS (
    SELECT 
        pa.id_producto,
        MAX(pa.fecha) AS fecha_adjunto   -- último adjunto activo por producto
    FROM slq_main.prod_adjuntos pa
    WHERE pa.activo = 1
    GROUP BY pa.id_producto
)

SELECT 
    p.id,
    p.created_at,

    -- updated_at: fecha del último updated en audit; si no hay, fallback a p.updated_at
    COALESCE(au.changed_at, p.updated_at) AS updated_at,

    -- usuario_crea: si hay audit de creación, ese; si no, usuario_registro
    COALESCE(u_crea.name, u_reg.name) AS usuario_crea,

    -- usuario_actualiza: último updated sobre ProdProducto
    u_upd.name AS usuario_actualiza,

    -- fecha del último adjunto activo
    adj.fecha_adjunto

FROM slq_main.prod_productos p

-- CREADOR
LEFT JOIN audit_crea ac
    ON ac.auditable_id = p.id
    AND ac.rn = 1
LEFT JOIN slq_main.sys_users u_crea
    ON u_crea.id = ac.user_id

-- FALLBACK: usuario_registro
LEFT JOIN slq_main.sys_users u_reg
    ON u_reg.id = p.usuario_registro

-- ÚLTIMO UPDATE DESDE AUDIT
LEFT JOIN audit_upd au
    ON au.auditable_id = p.id
    AND au.rn = 1
LEFT JOIN slq_main.sys_users u_upd
    ON u_upd.id = au.user_id

-- ADJUNTOS (último activo por producto)
LEFT JOIN adjuntos adj
    ON adj.id_producto = p.id

WHERE 
    p.modulo_creacion = 'productos'
    AND p.bloqueo IN (1, 2)
    AND p.activo = 1
""")

with engine_srv.connect() as conn:
    productos = pd.read_sql(query, conn)

productos


,id,created_at,updated_at,usuario_crea,usuario_actualiza,fecha_adjunto
0,6,2019-02-12 12:36:29,2025-03-23 11:49:57,None,Katherine Forero,2021-03-08
1,8,2019-02-12 12:36:30,2024-10-28 13:40:15,None,Katherine Forero,2024-09-03
2,9,2019-02-12 12:36:30,2023-06-26 09:13:36,None,Alejandro Puccini,2020-10-29
3,14,2019-02-12 12:36:30,2023-03-23 17:40:42,None,Liliana Lopez,2021-07-09
4,15,2019-02-12 12:36:30,2024-12-22 14:59:46,None,Leonardo Guzman,2024-03-20
...,...,...,...,...,...,...
29283,69264,2025-12-01 10:55:31,2025-12-01 11:04:42,Claudia Marcela Cortes Prieto,Claudia Marcela Cortes Prieto,2015-08-21
29284,69267,2025-12-01 11:58:07,2025-12-01 12:06:45,Angel Orozco,None,2020-07-24
29285,69268,2025-12-01 12:00:56,2025-12-01 12:05:57,Angel Orozco,None,2019-04-10
29286,69269,2025-12-01 12:02:44,2025-12-01 12:05:12,Angel Orozco,None,2023-12-02


In [10]:
productos.to_csv("productos.csv")

# Consultoría